# Simple Language Model with RNN

RNN을 이용하여 간단한 language model을 학습시켜 보겠습니다.

In [1]:
## library import
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)
print(keras.__version__)

2.0.0-beta1
2.2.4-tf


## Training Sentence

In [2]:
## 학습시킬 문장
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

## index를 주면 charcter로 바꿔주는 list
idx2char = list(set(sentence))
## character를 주면 index로 바꿔주는 dictionary
char2idx = {w: i for i, w in enumerate(idx2char)}

In [3]:
idx2char

['n',
 'u',
 'i',
 'h',
 ' ',
 'w',
 '.',
 'm',
 'b',
 ',',
 'o',
 'e',
 'c',
 's',
 'f',
 'l',
 'r',
 't',
 'y',
 'a',
 'p',
 'k',
 'g',
 'd',
 "'"]

In [4]:
char2idx

{'n': 0,
 'u': 1,
 'i': 2,
 'h': 3,
 ' ': 4,
 'w': 5,
 '.': 6,
 'm': 7,
 'b': 8,
 ',': 9,
 'o': 10,
 'e': 11,
 'c': 12,
 's': 13,
 'f': 14,
 'l': 15,
 'r': 16,
 't': 17,
 'y': 18,
 'a': 19,
 'p': 20,
 'k': 21,
 'g': 22,
 'd': 23,
 "'": 24}

## Hyper Parameters

In [5]:
data_dim = len(idx2char)
hidden_size = len(idx2char)
num_classes = len(idx2char)
sequence_length = 10  # Any arbitrary number
learning_rate = 0.1
training_epochs = 500
print(num_classes)

25


## Dataset

In [6]:
dataX = []
dataY = []
for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    print(i, x_str, '->', y_str)

    x = [char2idx[c] for c in x_str]  # x str to index
    y = [char2idx[c] for c in y_str]  # y str to index

    dataX.append(x)
    dataY.append(y)

batch_size = len(dataX)

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [7]:
dataX = np.array(to_categorical(dataX, num_classes))
dataY = np.array(to_categorical(dataY, num_classes))
print(dataX.shape, dataY.shape)
print(batch_size)

(170, 10, 25) (170, 10, 25)
170


In [8]:
train_dataset = tf.data.Dataset.from_tensor_slices((dataX, dataY)).shuffle(
                buffer_size=1000).prefetch(buffer_size=batch_size).batch(batch_size).repeat()

## Model Function

In [9]:
def create_model():
    model = keras.Sequential()
    model.add(layers.LSTM(units=hidden_size, return_sequences=True,
                                     input_shape=(dataX.shape[1],dataX.shape[2])))
    model.add(layers.LSTM(units=hidden_size, return_sequences=True))
    model.add(layers.Dense(units=num_classes, activation='softmax'))
    return model

In [10]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10, 25)            5100      
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 25)            5100      
_________________________________________________________________
dense (Dense)                (None, 10, 25)            650       
Total params: 10,850
Trainable params: 10,850
Non-trainable params: 0
_________________________________________________________________


## Loss & Optimizer

In [11]:
def loss(labels, logits):
    return keras.losses.categorical_crossentropy(labels, logits)

def adam_opt(learning_rate):
    return keras.optimizers.Adam(learning_rate)

## Model Compile

In [12]:
model.compile(optimizer=adam_opt(learning_rate),
              loss=loss)

## Training

In [13]:
model.fit(train_dataset, epochs=training_epochs,
                    steps_per_epoch=dataX.shape[0]//batch_size)

W0824 02:45:28.669958 11096 deprecation.py:323] From C:\Users\jwlee\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/500
1/1 [==============================] - 2s 2s/step - loss: 3.2188
Epoch 2/500
1/1 [==============================] - 0s 9ms/step - loss: 3.0053
Epoch 3/500
1/1 [==============================] - 0s 10ms/step - loss: 3.2812
Epoch 4/500
1/1 [==============================] - 0s 8ms/step - loss: 2.9679
Epoch 5/500
1/1 [==============================] - 0s 8ms/step - loss: 2.8929
Epoch 6/500
1/1 [==============================] - 0s 11ms/step - loss: 2.9012
Epoch 7/500
1/1 [==============================] - 0s 10ms/step - loss: 2.8685
Epoch 8/500
1/1 [==============================] - 0s 9ms/step - loss: 2.8389
Epoch 9/500
1/1 [==============================] - 0s 9ms/step - loss: 2.8597
Epoch 10/500
1/1 [==============================] - 0s 8ms/step - loss: 2.7804
Epoch 11/500
1/1 [==============================] - 0s 7ms/step - loss: 2.7606
Epoch 12/500
1/1 [==============================] - 0s 8ms/step - loss: 2.7140
Epoch 13/500
1/1 [==============================] - 0s 9ms/

Epoch 105/500
1/1 [==============================] - 0s 8ms/step - loss: 0.2375
Epoch 106/500
1/1 [==============================] - 0s 10ms/step - loss: 0.2376
Epoch 107/500
1/1 [==============================] - 0s 7ms/step - loss: 0.2372
Epoch 108/500
1/1 [==============================] - 0s 7ms/step - loss: 0.2369
Epoch 109/500
1/1 [==============================] - 0s 9ms/step - loss: 0.2369
Epoch 110/500
1/1 [==============================] - 0s 10ms/step - loss: 0.2365
Epoch 111/500
1/1 [==============================] - 0s 8ms/step - loss: 0.2364
Epoch 112/500
1/1 [==============================] - 0s 7ms/step - loss: 0.2363
Epoch 113/500
1/1 [==============================] - 0s 7ms/step - loss: 0.2360
Epoch 114/500
1/1 [==============================] - 0s 9ms/step - loss: 0.2358
Epoch 115/500
1/1 [==============================] - 0s 8ms/step - loss: 0.2355
Epoch 116/500
1/1 [==============================] - 0s 8ms/step - loss: 0.2354
Epoch 117/500
1/1 [===================

Epoch 208/500
1/1 [==============================] - 0s 7ms/step - loss: 0.2305
Epoch 209/500
1/1 [==============================] - 0s 9ms/step - loss: 0.2305
Epoch 210/500
1/1 [==============================] - 0s 7ms/step - loss: 0.2304
Epoch 211/500
1/1 [==============================] - 0s 8ms/step - loss: 0.2305
Epoch 212/500
1/1 [==============================] - 0s 7ms/step - loss: 0.2305
Epoch 213/500
1/1 [==============================] - 0s 8ms/step - loss: 0.2304
Epoch 214/500
1/1 [==============================] - 0s 9ms/step - loss: 0.2303
Epoch 215/500
1/1 [==============================] - 0s 7ms/step - loss: 0.2303
Epoch 216/500
1/1 [==============================] - 0s 9ms/step - loss: 0.2303
Epoch 217/500
1/1 [==============================] - 0s 8ms/step - loss: 0.2303
Epoch 218/500
1/1 [==============================] - 0s 7ms/step - loss: 0.2303
Epoch 219/500
1/1 [==============================] - 0s 9ms/step - loss: 0.2303
Epoch 220/500
1/1 [=====================

Epoch 311/500
1/1 [==============================] - 0s 8ms/step - loss: 0.2294
Epoch 312/500
1/1 [==============================] - 0s 10ms/step - loss: 0.2294
Epoch 313/500
1/1 [==============================] - 0s 7ms/step - loss: 0.2294
Epoch 314/500
1/1 [==============================] - 0s 8ms/step - loss: 0.2295
Epoch 315/500
1/1 [==============================] - 0s 8ms/step - loss: 0.2295
Epoch 316/500
1/1 [==============================] - 0s 7ms/step - loss: 0.2295
Epoch 317/500
1/1 [==============================] - 0s 7ms/step - loss: 0.2295
Epoch 318/500
1/1 [==============================] - 0s 8ms/step - loss: 0.2296
Epoch 319/500
1/1 [==============================] - 0s 8ms/step - loss: 0.2297
Epoch 320/500
1/1 [==============================] - 0s 8ms/step - loss: 0.2298
Epoch 321/500
1/1 [==============================] - 0s 7ms/step - loss: 0.2300
Epoch 322/500
1/1 [==============================] - 0s 8ms/step - loss: 0.2299
Epoch 323/500
1/1 [====================

Epoch 414/500
1/1 [==============================] - 0s 7ms/step - loss: 0.2291
Epoch 415/500
1/1 [==============================] - 0s 9ms/step - loss: 0.2291
Epoch 416/500
1/1 [==============================] - 0s 8ms/step - loss: 0.2291
Epoch 417/500
1/1 [==============================] - 0s 7ms/step - loss: 0.2291
Epoch 418/500
1/1 [==============================] - 0s 8ms/step - loss: 0.2291
Epoch 419/500
1/1 [==============================] - 0s 8ms/step - loss: 0.2291
Epoch 420/500
1/1 [==============================] - 0s 7ms/step - loss: 0.2291
Epoch 421/500
1/1 [==============================] - 0s 10ms/step - loss: 0.2291
Epoch 422/500
1/1 [==============================] - 0s 8ms/step - loss: 0.2291
Epoch 423/500
1/1 [==============================] - 0s 7ms/step - loss: 0.2291
Epoch 424/500
1/1 [==============================] - 0s 8ms/step - loss: 0.2291
Epoch 425/500
1/1 [==============================] - 0s 9ms/step - loss: 0.2291
Epoch 426/500
1/1 [====================

## Checking the Test Result

In [14]:
results = model.predict(dataX, steps=1)
for j, result in enumerate(results):
    index = np.argmax(result, axis=1)
    if j is 0:  # print all for the first result to make a sentence
        print(''.join([idx2char[t] for t in index]), end='')
    else:
        print(idx2char[index[-1]], end='')

m you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.

# Using GradientTape

In [15]:
train_dataset = tf.data.Dataset.from_tensor_slices((dataX, dataY)).shuffle(
                buffer_size=100000).prefetch(buffer_size=batch_size).batch(batch_size)

In [16]:
model = create_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 10, 25)            5100      
_________________________________________________________________
lstm_3 (LSTM)                (None, 10, 25)            5100      
_________________________________________________________________
dense_1 (Dense)              (None, 10, 25)            650       
Total params: 10,850
Trainable params: 10,850
Non-trainable params: 0
_________________________________________________________________


In [17]:
@tf.function
def loss_fn(model, dataX, dataY):
    logits = model(dataX, training=True)
    loss = tf.reduce_mean(keras.losses.categorical_crossentropy(labels, logits))    
    return loss 

In [18]:
@tf.function
def train(model, dataX, dataY):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, dataX, dataY)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

In [19]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

In [20]:
# train my model
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_loss = 0.    
    train_step = 0    
    
    for images, labels in train_dataset:
        train(model,images, labels)
        loss = loss_fn(model, images, labels)        
        avg_loss = avg_loss + loss        
        train_step += 1
    avg_loss = avg_loss / train_step    

    print('Epoch:', '{}'.format(epoch + 1), 'loss =', '{:.8f}'.format(avg_loss))


print('Learning Finished!')

Learning started. It takes sometime.
Epoch: 1 loss = 3.07759881
Epoch: 2 loss = 3.06852317
Epoch: 3 loss = 2.85521865
Epoch: 4 loss = 2.88638926
Epoch: 5 loss = 2.85205817
Epoch: 6 loss = 2.83029437
Epoch: 7 loss = 2.80191469
Epoch: 8 loss = 2.77385449
Epoch: 9 loss = 2.72482085
Epoch: 10 loss = 2.64846969
Epoch: 11 loss = 2.58146715
Epoch: 12 loss = 2.52702236
Epoch: 13 loss = 2.48046231
Epoch: 14 loss = 2.36589098
Epoch: 15 loss = 2.28911567
Epoch: 16 loss = 2.20882058
Epoch: 17 loss = 2.10518885
Epoch: 18 loss = 2.02794337
Epoch: 19 loss = 1.93242681
Epoch: 20 loss = 1.82313466
Epoch: 21 loss = 1.71803606
Epoch: 22 loss = 1.62320411
Epoch: 23 loss = 1.52482915
Epoch: 24 loss = 1.43298185
Epoch: 25 loss = 1.34930778
Epoch: 26 loss = 1.25959277
Epoch: 27 loss = 1.18905556
Epoch: 28 loss = 1.09541285
Epoch: 29 loss = 1.03327262
Epoch: 30 loss = 0.96477699
Epoch: 31 loss = 0.90351033
Epoch: 32 loss = 0.83492243
Epoch: 33 loss = 0.77805585
Epoch: 34 loss = 0.73048568
Epoch: 35 loss = 0.6

Epoch: 290 loss = 0.22939564
Epoch: 291 loss = 0.22957191
Epoch: 292 loss = 0.22955830
Epoch: 293 loss = 0.22948480
Epoch: 294 loss = 0.22970267
Epoch: 295 loss = 0.22990137
Epoch: 296 loss = 0.22993030
Epoch: 297 loss = 0.23008181
Epoch: 298 loss = 0.23008305
Epoch: 299 loss = 0.22974060
Epoch: 300 loss = 0.22946411
Epoch: 301 loss = 0.22938548
Epoch: 302 loss = 0.22936256
Epoch: 303 loss = 0.22949858
Epoch: 304 loss = 0.22964962
Epoch: 305 loss = 0.22958548
Epoch: 306 loss = 0.22941774
Epoch: 307 loss = 0.22932495
Epoch: 308 loss = 0.22930190
Epoch: 309 loss = 0.22931379
Epoch: 310 loss = 0.22935778
Epoch: 311 loss = 0.22937642
Epoch: 312 loss = 0.22930758
Epoch: 313 loss = 0.22921930
Epoch: 314 loss = 0.22922346
Epoch: 315 loss = 0.22927731
Epoch: 316 loss = 0.22928859
Epoch: 317 loss = 0.22925732
Epoch: 318 loss = 0.22921351
Epoch: 319 loss = 0.22918625
Epoch: 320 loss = 0.22918084
Epoch: 321 loss = 0.22919035
Epoch: 322 loss = 0.22920689
Epoch: 323 loss = 0.22921298
Epoch: 324 los

In [21]:
results = model(dataX, training=False)

for j, result in enumerate(results):
    index = np.argmax(result, axis=1)
    if j is 0:  # print all for the first result to make a sentence
        print(''.join([idx2char[t] for t in index]), end='')
    else:
        print(idx2char[index[-1]], end='')

t you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.